# Dynamic Data Ingestion to Bronze Layer

## Environment Configuration

In [0]:
from pyspark.sql.utils import AnalysisException
from src.utils.udfs import functions_for_df_structure_management as ffdsm

In [0]:
dbutils.widgets.text("file_name", "")
file_name = dbutils.widgets.get("file_name")

## Raw Data Ingestion

In [0]:
df_bronze = spark.read.format("csv").option("header", "true").option("delimiter", ";").load(f"/Volumes/brazilian_tax_big_numbers/data_sources/data_sources/{file_name}")

## Standardization of Column Names

In [0]:
columns_df_bronze = [ffdsm.normalize_column_name(col) for col in df_bronze.columns]

In [0]:
df_bronze = df_bronze.toDF(*columns_df_bronze)

In [0]:
table_name = ffdsm.normalize_file_name(file_name)

## Save Table as Delta in Bronze Layer

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS brazilian_tax_big_numbers.bronze_layer")

In [0]:
error = None

try:
    df_bronze.write \
        .mode("overwrite") \
        .saveAsTable(f"brazilian_tax_big_numbers.bronze_layer.delta_{table_name}")
    error = None
except Exception as e:
    error = str(e)
    print(error)